# ETL Processes

This notebook contains the elements to construct ETL process for each of the tables in etl.py file to load the dataset.

In [9]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [10]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

OperationalError: connection to server at "127.0.0.1", port 5432 failed: FATAL:  password authentication failed for user "student"


In [11]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    return all_files

# Process song_data

Performing ETL on song_data to create song and artist dimension tables.

In [12]:
song_files = get_files('data/song_data/')

In [13]:
filepath = song_files[0]
print(filepath)

d:\Data-Engineering-Nanodegree\Data Modeling with Postgres\data\song_data\A\A\A\TRAAAAW128F429D538.json


In [14]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


# #1: songs table

## Extract Data for Songs Table
* Select columns for song_id, title, artist_id, year, and duration
* Use df.values to select just the values from the dataframe
* Index to select the first (only) record in the dataframe
* Convert the array to a list and set it to song_data   

In [19]:
num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = df.values[0]

In [20]:
song_data = [ song_id, title, artist_id, year, duration]

song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

<b>Insert record into Song Table</b>

Implement the song_table_insert query in sql_queries.py and run the cell below to insert a record for this song into the songs table. Remember to run create_table.py before running the cell below to ensure you have created/resetted the songs table in the sparkify database.